In [1]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import seaborn as sns
import random

def configure_model(model):
    model.Params.OutputFlag = 0
    model.Params.TimeLimit = 100
    model.Params.MIPGap = 0.05
    model.Params.Presolve = 2
    model.Params.Method = 1        
    model.Params.MIPFocus = 1        
    model.Params.Heuristics = 0.8   
    model.Params.Threads = 8

In [2]:
# Parameters
m = 8
K = 5
L_levels = [20, 15, 10, 5, 1]  
E_levels = [13, 28, 38, 45, 120] 
E_PU_max = [8000] * m # limit 650
CPU_capacity = [3000] * m
f_v_levels = [30, 40, 60, 80, 100]
P_idle = [18] * m
P_max = [36] * m
T_total = 300
total_tasks = 150
from collections import Counter

# Task chain definitions
chain_tasks = {
    0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    1: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
    2: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
    3: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
    4: [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
    5: [50, 51, 52, 53, 54, 55, 56, 57],
    6: [58, 59, 60],
    7: [61, 62, 63],
    8: [64, 65, 66],
    9: [67, 68, 69],
    10: [70, 71, 72],
    11: [73, 74, 75],
    12: [76, 77, 78],
    13: [79, 80, 81],
    14: [82, 83, 84],
    15: [85, 86, 87],
    16: [88, 89, 90],
    17: [91, 92, 93],
    18: [94, 95, 96],
    19: [97, 98, 99]
}

num_chains = len(chain_tasks)
n = max(max(v) for v in chain_tasks.values()) + 1  # Total number of tasks
L_max = [len(chain_tasks[k]) * 8 for k in chain_tasks]  # Max delay per chain




def run_gurobi_model(alpha_1, alpha_2, alpha_3, baseline=False):
    model = gp.Model("MAF_Optimization")
    configure_model(model)

    delta = model.addVars(n, vtype=GRB.BINARY, name="delta")
    w = model.addVars(n, m, vtype=GRB.BINARY, name="w")
    y = model.addVars(n, K, vtype=GRB.BINARY, name="y")
    z = model.addVars(num_chains, vtype=GRB.BINARY, name="z")
    E_PU = model.addVars(m, name="E_PU")

    if baseline:
        for i in range(n):
            model.addConstr(y[i, K - 1] == delta[i])
            for k in range(K - 1):
                model.addConstr(y[i, k] == 0)

    for i in range(n):
        model.addConstr(gp.quicksum(w[i, j] for j in range(m)) == delta[i])
        model.addConstr(gp.quicksum(y[i, k] for k in range(K)) == delta[i])
        for k in range(K):
            model.addConstr(y[i, k] <= delta[i])  # Ensure y[i,k]=0 if task is inactive

    for k in range(num_chains):
        model.addConstr(gp.quicksum(delta[i] for i in chain_tasks[k]) == z[k] * len(chain_tasks[k]))
        delay_chain = gp.quicksum(y[i, k_] * L_levels[k_] for i in chain_tasks[k] for k_ in range(K))
        model.addConstr(delay_chain <= L_max[k])

    for j in range(m):
        model.addConstr(
            gp.quicksum(w[i, j] * gp.quicksum(y[i, k] * f_v_levels[k] for k in range(K)) for i in range(n)) <= CPU_capacity[j]
        )
        active_time = gp.quicksum(w[i, j] * gp.quicksum(y[i, k] * L_levels[k] for k in range(K)) for i in range(n))
        U_j = active_time / T_total
        E_PU_expr = P_idle[j] * (T_total - active_time) + (P_max[j] - P_idle[j]) * U_j * T_total
        model.addConstr(E_PU[j] == E_PU_expr)
        model.addConstr(E_PU[j] <= E_PU_max[j])

    total_task_energy = gp.quicksum(y[i, k] * E_levels[k] for i in range(n) for k in range(K))
    total_chain_accept = gp.quicksum(z[k] for k in range(num_chains))
    total_pu_energy = gp.quicksum(E_PU[j] for j in range(m))

    model.setObjective(
        alpha_1 * total_chain_accept - alpha_2 * total_task_energy - alpha_3 * total_pu_energy,
        GRB.MAXIMIZE
    )

    start_time = time.time()
    model.optimize()
    end_time = time.time()

    if model.status != GRB.OPTIMAL:
        print("❌ Model optimization failed. Skipping variable access.")
        return None

    print(f"⏱️ Gurobi Runtime: {end_time - start_time:.2f} seconds")
    print("\n📋 Chain Activation Overview:")
    for k in range(num_chains):
        status = "✅ Activated" if z[k].X > 0.5 else "❌ Not activated"
        task_list = chain_tasks[k]
        print(f"Chain {k}: {status}, Tasks: {task_list}")

    accepted_chains = [k for k in range(num_chains) if z[k].X > 0.5]
    pu_assignment = [j for i in range(n) for j in range(m) if w[i, j].X > 0.5]
    pu_task_map = {}
    for task_id, pu_id in enumerate(pu_assignment):
        pu_task_map.setdefault(pu_id, []).append(task_id)

    task_energy_levels = [max((y[i, k].X, k) for k in range(K))[1] if delta[i].X > 0.5 else -1 for i in range(n)]

    return {
        "Mode": "Baseline" if baseline else "Non-Baseline",
        "Objective Value": model.ObjVal,
        "Accepted Chains": len(accepted_chains),
        "Total PU Energy": sum(E_PU[j].X for j in range(m)),
        "Total Task Energy": sum(y[i, k].X * E_levels[k] for i in range(n) for k in range(K)),
        "Runtime": model.Runtime,
        "PU Task Assignment": pu_task_map,
        "Task Energy Levels": task_energy_levels,
        "Activated Chain List": [1 if z[k].X > 0.5 else 0 for k in range(num_chains)]
    }

In [4]:
alpha_combinations = [
    (1000, 1, 1),
    (1000, 5, 5),
    (1000, 10, 10),
    (500, 5, 10),
    (100, 10, 20),
    (50, 10, 50)
]
E_PU_max_values = [5000, 6000, 7000, 8000, 9000]

results = []

for emax in E_PU_max_values:
    E_PU_max = [emax] * m  # 设置 PU 限制
    for (a1, a2, a3) in alpha_combinations:
        print(f"🧪 Running: E_PU_max={emax}, α=({a1},{a2},{a3})")
        res = run_gurobi_model(alpha_1=a1, alpha_2=a2, alpha_3=a3, baseline=False)

        if res is not None:
            res.update({
                "E_PU_max": emax,
                "alpha_1": a1,
                "alpha_2": a2,
                "alpha_3": a3
            })
            results.append(res)
        else:
            print(f"⚠️ Gurobi failed for α=({a1},{a2},{a3}) at E_PU_max={emax}")


🧪 Running: E_PU_max=5000, α=(1000,1,1)
❌ Model optimization failed. Skipping variable access.
⚠️ Gurobi failed for α=(1000,1,1) at E_PU_max=5000
🧪 Running: E_PU_max=5000, α=(1000,5,5)
❌ Model optimization failed. Skipping variable access.
⚠️ Gurobi failed for α=(1000,5,5) at E_PU_max=5000
🧪 Running: E_PU_max=5000, α=(1000,10,10)
❌ Model optimization failed. Skipping variable access.
⚠️ Gurobi failed for α=(1000,10,10) at E_PU_max=5000
🧪 Running: E_PU_max=5000, α=(500,5,10)
❌ Model optimization failed. Skipping variable access.
⚠️ Gurobi failed for α=(500,5,10) at E_PU_max=5000
🧪 Running: E_PU_max=5000, α=(100,10,20)
❌ Model optimization failed. Skipping variable access.
⚠️ Gurobi failed for α=(100,10,20) at E_PU_max=5000
🧪 Running: E_PU_max=5000, α=(50,10,50)
❌ Model optimization failed. Skipping variable access.
⚠️ Gurobi failed for α=(50,10,50) at E_PU_max=5000
🧪 Running: E_PU_max=6000, α=(1000,1,1)
⏱️ Gurobi Runtime: 0.20 seconds

📋 Chain Activation Overview:
Chain 0: ✅ Activated, T